In [4]:
#构造卷积神经网络之LeNet
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

# 配置神经网络的参数
INPUT_NODE = 784
OUTPUT_NODE = 10

IMAGE_SIZE = 28
NUM_CHANNELS = 1
NUM_LABELS = 10

# 第一层卷积神经网络的尺寸和深度
CONV1_DEEP = 32
CONV1_SIZE = 5

# 第二层卷积神经网络的尺寸和深度
CONV2_DEEP=64
CONV2_SIZE=5

#全连接节点的个数
FC_SIZE=512

BATCH_SIZE=80
LEARNING_RATE_BASE = 0.001
LEARNING_RATE=0.96
LEARNING_RRATE_DECAY = 0.99
MOVING_AVERAGE_DECAY = 0.99
REGULARRIZATON_RATE = 0.0001
TRAINING_STEEPS =3000
# 模型保存的路径和文件名
MODEL_SAVE_PATH ='Mnistdata_Model'
MODEL_NAME ='model.ckpt'

# 定义神经网络向前传播的过程 添加一个新参数train,用于区分训练过程好人测试过程 使用了drouout方法
def inference(input_tensor,train,regularizer):
    with tf.variable_scope('layer1-conv1'):
        conv1_weight=tf.get_variable('weight',[CONV1_SIZE,CONV1_SIZE,NUM_CHANNELS,CONV1_DEEP],
                                     initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv1_bias=tf.get_variable('bias',[CONV1_DEEP],initializer=tf.constant_initializer(0.0))
        
#         使用边长为5，深度为32的滤波器，且移动步长为1，并使用全0填充
        conv1 = tf.nn.conv2d(input_tensor,conv1_weight,strides=[1,1,1,1],padding='SAME')
        
        relu1=tf.nn.relu(tf.nn.bias_add(conv1,conv1_bias))
        
#         实现第二层的池化层、
    with tf.name_scope('layer2-pool'):
        pool1=tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
            
#     实现第三层向前传播
    with tf.variable_scope('layer3-conv2'):
        conv2_weights=tf.get_variable('weight',[CONV2_SIZE,CONV2_SIZE,CONV1_DEEP,CONV2_DEEP],
                              initializer=tf.truncated_normal_initializer(0.1))
        conv2_bias=tf.get_variable('bias',[CONV2_DEEP],initializer=tf.constant_initializer(0.0))

#         使用边长为5，深度为64的过滤器，移动步长为1，使用全0填充
        conv2=tf.nn.conv2d(pool1,conv2_weights,strides=[1,1,1,1],padding='SAME')
        relu2=tf.nn.relu(tf.nn.bias_add(conv2,conv2_bias))
        
#     实现第四层的前向传播过程
    with tf.name_scope('layer4-pool'):
        pool2=tf.nn.max_pool(relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        
#     实现第五层的全连接，全连接的输入格式为向量
    pool_shape=pool2.get_shape().as_list()
#     计算将矩阵拉伸后的长度，这个长度就是矩阵长宽和深度的乘积 pool_shape[0]为一个batch中数据的个数
    nodes=pool_shape[1]*pool_shape[2]*pool_shape[3]
#     将第四层输出为一个batch的向量
    reshaped=tf.reshape(pool2,[pool_shape[0],nodes])
    
#     第五层全连接的变量实现前向传播，这一层的输入时拉直后的一组向量，向量长度是3136，输出是一组长度为512的变量 引入dropout
    with tf.variable_scope('layer5-fcn1'):
        fc1_weights=tf.get_variable('weight',[nodes,FC_SIZE],initializer=tf.truncated_normal_initializer(stddev=0.1))
        fc1_bias=tf.get_variable('bias',[FC_SIZE],initializer=tf.constant_initializer(0.0))
#         只有全连接的权重需要正则化
        if regularizer != None:
            tf.add_to_collection('losses',regularizer(fc1_weights))
        fc1=tf.nn.relu(tf.matmul(reshaped,fc1_weights)+fc1_bias)
        if train: 
            fc1=tf.nn.dropout(fc1,0.5)
            
#         第六层全连接的变量实现前向传播，这一层的输入为一组长度为512的向量，输出为一组长度为10的变量，通过softmax得到分类结果
    with tf.variable_scope('layer6-fc2'):
        fc2_weights=tf.get_variable('weight',[FC_SIZE,NUM_LABELS],initializer=tf.truncated_normal_initializer(0.1))
        fc2_bias=tf.get_variable('bias',[NUM_LABELS],initializer=tf.constant_initializer(0.1))
        if regularizer != None:
            tf.add_to_collection('losses',regularizer(fc2_weights))
        logit=tf.matmul(fc1,fc2_weights)+fc2_bias
        
    return logit
        
# 每10秒加载一次最新的模型，并在测试
EVAL_INTERVAL_SECS = 10
def evaluate(mnist):
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32,[None,INPUT_NODE],name='x-input')
        x_image =tf.reshape(x,[BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,NUM_CHANNELS])
        y_=tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='y-input')
        
#         validate_feed = {x:mnist.validation.images,y_:mnist.validation.labels}
        #前向传播
        regularizer = tf.contrib.layers.l2_regularizer(REGULARRIZATON_RATE)
        y = inference(x_image,False,regularizer)

        global_step = tf.Variable(0,trainable=False)
        variable_average = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
        variable_averages_op = variable_average.apply(tf.trainable_variables())
        
        cross_entropy =tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
        # 所有样例的交叉熵平均值
        cross_entropy_mean = tf.reduce_mean(cross_entropy)
        tf.add_to_collection('losses',cross_entropy_mean)
        loss =tf.add_n(tf.get_collection('losses'))
        learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,mnist.train.num_examples/BATCH_SIZE,LEARNING_RRATE_DECAY)
        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
    
        with tf.control_dependencies([train_step,variable_averages_op]):
            train_op = tf.no_op(name='train')
            
        saver = tf.train.Saver()      
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            for i in range(TRAINING_STEEPS):
                xs,ys = mnist.train.next_batch(BATCH_SIZE)
                _,loss_value,step = sess.run([train_op,loss,global_step],feed_dict={x:xs,y_:ys})

                if i%100 ==0:
                    print('After %d training step(s),loss on training batch is %g'%(step,loss_value))
                    saver.save(sess,os.path.join(MODEL_SAVE_PATH,MODEL_NAME),global_step=global_step)

def main():
    mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)
    evaluate(mnist)
    
if __name__ == '__main__':
    main()


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
After 1 training step(s),loss on training batch is 999.118
After 101 training step(s),loss on training batch is 2.91828
After 201 training step(s),loss on training batch is 2.98742
After 301 training step(s),loss on training batch is 2.77942
After 401 training step(s),loss on training batch is 3.10567
After 501 training step(s),loss on training batch is 2.87862
After 601 training step(s),loss on training batch is 2.62748
After 701 training step(s),loss on training batch is 2.84695
After 801 training step(s),loss on training batch is 3.10326
After 901 training step(s),loss on training batch is 2.57894
After 1001 training step(s),loss on training batch is 2.72898
After 1101 training step(s),loss on training batch is 2.43144
After 1201 training step(s),loss on training batch is 2.61999
After 1301 t

In [5]:
#加入dropout后的卷积神经网络
import tensorflow as t
from tensorflow.examples.tutorials.mnist import input_data
import os
import numpy as np

INPUT_NODE = 784  #输入层的节点数，这里是图片的像素
OUTPUT_NODE = 10  # 输出层的节点数，这里是类别的数目

IMAGE_SIZE = 28   # 图片大小
NUM_CHANNELS = 1  # 数字通道
NUM_LABELS = 10   # 数字类别

# 第一层卷积层的尺寸和深度
CONV1_DEEP = 32 
CONV1_SIZE = 5
# 第二层卷积层的尺寸和深度
CONV2_DEEP = 64
CONV2_SIZE = 5
# 全连接层的节点个数
FC_SIZE = 512

BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.01
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 3000
MOVING_AVERAGE_DECAY = 0.99

# #### 2. 定义前向传播的过程

# 这里添加了一个新的参数train，用于区分训练过程和测试过程。在这个过程中将用到dropout方法，该方法可以进一步提升
# 模型的可靠性并防止过拟合，dropout过程只在训练过程中使用。
def inference(input_tensor, train, regularizer):

    # 声明第一层卷积层的变量并实现前向传播过程。通过使用不同的命名空间来隔离不同层的变量，这可以让每一层中的
    # 变量命名只需要考虑在当前层的作用，而不用担心重名的问题。和标准的LeNet-5模型不大一样，这里定义的卷积层输入
    # 为28x28x1的原始MNIST图片像素。因为卷积层中使用了全0填充，所以输出为28x28x32的矩阵。
    with tf.variable_scope('layer1-conv1'): # 在名字为layer1-conv1的命名空间内创建名字为weight的变量
        conv1_weights = tf.get_variable(
            "weight", [CONV1_SIZE, CONV1_SIZE, NUM_CHANNELS, CONV1_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1)) # 初始化满足正太分布的截随机数
        conv1_biases = tf.get_variable("bias", [CONV1_DEEP], initializer=tf.constant_initializer(0.0))
        # 使用边长为5，深度为32的过滤器，过滤器移动的步长为1，且使用全0填充。tf.nn.conv2d函数的第一个输入为当前
        # 层的节点矩阵（这个矩阵是四维矩阵，第一维对应的是输入一个batch，后面三维对应一个节点矩阵）。第二个参数
        # 提供了卷积层的权重，第三个参数为不同维度上的步长（虽然提供的是一个长度为4的数组，但第一维和最后一维的数字一定要是1
        # 这是因为卷积层的步长只对矩阵的长和宽有效）。最后一个参数是填充，其中'SAME'表示全0填充，'VALLD'表示不添加。
        conv1 = tf.nn.conv2d(input_tensor, conv1_weights, strides=[1, 1, 1, 1], padding='SAME')
        # tf.nn.bias_add提供了一个方便的函数给每一个节点加上偏置项，注意这里不能直接使用加法，因为矩阵上的不同位置
        # 上的节点都需要加上同样的偏置项。然后再通过RELU激活函数完成去线性化。
        relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_biases))
    # 实现第二层池化层的前向层的传播过程。这里用的是最大池化层，池化层过滤器的边长为2，使用全0填充，并且移动的步长
    # 为2.这一层的输入是上一层的输出。也就是输入为28x28x32的矩阵，输出为14x14x32的矩阵。
    with tf.name_scope("layer2-pool1"):
        pool1 = tf.nn.max_pool(relu1, ksize = [1,2,2,1],strides=[1,2,2,1],padding="SAME")

    # 声明第三层卷积层的变量并实现前向传播过程。这一层的输入为14x14x32的矩阵，输出为14x14x64的矩阵。
    with tf.variable_scope("layer3-conv2"):
        conv2_weights = tf.get_variable(
            "weight", [CONV2_SIZE, CONV2_SIZE, CONV1_DEEP, CONV2_DEEP],
            initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv2_biases = tf.get_variable("bias", [CONV2_DEEP], initializer=tf.constant_initializer(0.0))
        # 使用边长为5，深度为64的过滤器，过滤器移动的步长为1，且使用全0填充。
        conv2 = tf.nn.conv2d(pool1, conv2_weights, strides=[1, 1, 1, 1], padding='SAME')
        relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))

    # 实现第四层池化层的前向传播过程。这一层和第二层的结构是一样的。这一层的输入为14x14x64的矩阵，输出为7x7x64
    # 的矩阵。
    with tf.name_scope("layer4-pool2"):
        pool2 = tf.nn.max_pool(relu2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        # 将第四层池化层的输出转化为第五层全连接层的输入格式。第四层的输出为7x7x64的矩阵，然而第五层全连接层的
        # 需要的输入格式为向量，所以这里需要将这个7x7x64的矩阵拉直成一个向量。pool2.get_shape函数可以得到第四层
        # 输出矩阵的维度而不需要自己去计算，注意因为每一层神经网络的输入输出都为一个batch的矩阵，所以这里得到的维度
        # 也包含了一个batch中数据的个数。
        pool_shape = pool2.get_shape().as_list()
        # 计算将矩阵拉直成向量之后的长度，这个长度就是矩阵长宽及深度的乘积。注意这里pool_shape[0]
        # 为一个batch中数据的个数。
        nodes = pool_shape[1] * pool_shape[2] * pool_shape[3]
        # 通过tf.reshape函数将第四层的输出变成一个batch的向量。
        reshaped = tf.reshape(pool2, [pool_shape[0], nodes])

    # 声明第五层全连接层的变量并实现前向传播过程。这一层的输入是拉直的一组向量，向量长度为3136，输出是一组长度为
    # 512的向量。这里引入dropout的概念。dropout在训练时会随机将部分节点的输出改为0，dropout可以避免过拟合问题从而
    # 可以使得模型在测试数据上的效果更好。dropout一般只在全连接层使用而不是卷积层或者池化层使用。
    with tf.variable_scope('layer5-fc1'):
        fc1_weights = tf.get_variable("weight", [nodes, FC_SIZE],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        # 只有全连接层的权重需要加入正则化
        if regularizer != None: 
            tf.add_to_collection('losses', regularizer(fc1_weights))
        fc1_biases = tf.get_variable("bias", [FC_SIZE], initializer=tf.constant_initializer(0.1))
        # 通过RELU激活函数完成去线性化。其中tf.matmul实现的是矩阵乘法
        fc1 = tf.nn.relu(tf.matmul(reshaped, fc1_weights) + fc1_biases)
        if train: 
            fc1 = tf.nn.dropout(fc1, 0.5)
    # 声明第六层全连接层的变量并实现前向传播过程。这一层的输入为一组长为512的向量，输出为一组长为10的向量。
    # 这一层的输出通过softmax之后就得到了最后的分类结果。
    with tf.variable_scope('layer6-fc2'):
        fc2_weights = tf.get_variable("weight", [FC_SIZE, NUM_LABELS],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None: tf.add_to_collection('losses', regularizer(fc2_weights))
        fc2_biases = tf.get_variable("bias", [NUM_LABELS], initializer=tf.constant_initializer(0.1))
        logit = tf.matmul(fc1, fc2_weights) + fc2_biases
    # 返回第六层的输出
    return logit


# #### 2. 定义训练过程

# In[ ]:

# 调整输入数据placeholder的格式，输入为一个四维的矩阵
def train(mnist):
    # 定义输入为4维矩阵的placeholder
    x = tf.placeholder(tf.float32, 
        [ BATCH_SIZE,  # 第一维表示一个batch中样例的个数 
            IMAGE_SIZE, # 第二维和第三维表示图片的尺寸
            IMAGE_SIZE,
            NUM_CHANNELS], # 第四维表示图片的深度，对于RGB格式的图片深度是5
        name='x-input')
    # 定义输出的placeholder
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')

    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    # 直接使用LeNet5_infernece.py中定义的前向传播过程
    y = inference(x,False,regularizer)
    global_step = tf.Variable(0, trainable=False)

    # 定义损失函数、学习率、滑动平均操作以及训练过程。
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY,
        staircase=True)

    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')

    # 初始化TensorFlow持久化类。
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        # 在训练过程中不再测试模型在验证数据上的表现，验证和测试的过程将会有一个独立的程序来完成。
        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)

            reshaped_xs = np.reshape(xs, (
                BATCH_SIZE,
                IMAGE_SIZE,
                IMAGE_SIZE,
                NUM_CHANNELS))
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: reshaped_xs, y_: ys})
            # 每1000轮输出一次当前的训练情况
            if i % 1000 == 0:
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))


# #### 3. 主程序入口

# In[ ]:

def main(argv=None):
    tf.reset_default_graph()
    mnist = input_data.read_data_sets(r"MNIST_data/", one_hot=True)
    train(mnist)

if __name__ == '__main__':
    main()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
After 1 training step(s), loss on training batch is 4.73072.
After 1001 training step(s), loss on training batch is 0.712054.
After 2001 training step(s), loss on training batch is 0.678772.


V1/a1:0
V1/a2:0
V2/a1:0
V2/a2:0
